# 감성분석(Sentiment Analysis)
- 문서의 주관적인 감성/의견/감정/기분 등을 파악하기 위해 다양한 분야에서 사용되고 있음
- 문서 내 텍스트가 나타내는 여러 가지 주관적인 단어와 문맥을 기반으로 감성(Sentitment) 수치를 계산하는 방법을 이용함. 이러한 감성지수는 긍정 감성 지수와 부정 감성 지수로 구성되며 이들 지수를 합산해 긍정 감성 또는 부정 감성을 결정함. 이는 머신러닝 관점에서 지도학습과 비지도 학습으로 나뉨
- 지도학습은 학습 데이터와 타겟 레이블 값을 기반으로 감성 분석 학습을 수행한 뒤 이를 기반으로 다른 데이터의 감성 분석을 예측하는 방법으로 일반적인 텍스트 기반의 분류와 거의 동일함
- 비지도학습은 Lexicon이라는 일종의 감성 어휘 사전을 이용함. Lexicon은 감성 분석을 위한 용어와 문맥에 대한 다양한 정보를 가지고 있으며 이를 이용해 문서의 긍정적, 부정적 감성 여부를 판단함

## 지도학습 기반 감성 분석 실습 -IMDB 영화평


In [2]:
import pandas as pd
review_df=pd.read_csv("C:/Users/MYCOM/data/labeledTrainData.tsv/labeledTrainData.tsv",header=0,sep='\t',quoting=3)
review_df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [3]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [4]:
import re
# <br> html 태그는 replace 함수로 공백으로 변환 
review_df['review']=review_df['review'].str.replace('<br />',' ')
review_df['review']=review_df['review'].apply(lambda x: re.sub('[^a-zA-Z]',' ',x))

In [5]:
from sklearn.model_selection import train_test_split
class_df=review_df['sentiment']
feature_df=review_df.drop(['id','sentiment'],axis=1,inplace=False)
X_train,X_test,y_train,y_test=train_test_split(feature_df,class_df,test_size=0.3,random_state=156)
X_train.shape,X_test.shape

((17500, 1), (7500, 1))

In [9]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score
# 스톱워드는 English, filtering, ngram 은 (1,2)로 설정해 CountVectorization 수행
# LogisticRegression의 C는 10으로 설정 
pipeline=Pipeline([
    ('cnt_vect',CountVectorizer(stop_words='english',ngram_range=(1,2))),
    ('lr_clf',LogisticRegression(C=10))])
# Pipeline 객체를 이용해 fit, predict 으로 학습/예측 수행 
# predict_proba는 roc_auc 때문에 수행
pipeline.fit(X_train['review'],y_train)
pred=pipeline.predict(X_test['review'])
pred_probas=pipeline.predict_proba(X_test['review'])[:,1]
print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test,pred),
                                                roc_auc_score(y_test,pred_probas)))

C:\Users\MYCOM\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


예측 정확도는 0.8860, ROC-AUC는 0.9503


In [10]:
# TF-IDF 벡터화를 적용해 다시 예측 성능을 측정해보기 
pipeline=Pipeline([
    ('tfidf_vect',TfidfVectorizer(stop_words='english',ngram_range=(1,2))),
    ('lr_clf',LogisticRegression(C=10))])
pipeline.fit(X_train['review'],y_train)
pred=pipeline.predict(X_test['review'])
pred_probas=pipeline.predict_proba(X_test['review'])[:,1]
print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test,pred),
                                                roc_auc_score(y_test,pred_probas)))

예측 정확도는 0.8936, ROC-AUC는 0.9598


# 비지도학습 기반 감성 분석
## Lexicon을 기반으로 
- 주로 감성만을 분석하기 위해 지원하는 감성 어휘 사전
- 감성 사전을 구현한 대표격은 NLTK 패키지,여기는 많은 서브 모듈을 가지고 잇으며 그 중 감성 사전인 Lexicon 모듈도 포함되어 있음
- NLP 패키지의 WordNet이란 방대한 영어 어휘 사전으로 단순한 어휘 사전이 아닌 시맨틱 분석을 제공하는 어휘 사전임 # 시멘틱이란 문맥상 의미란 뜻
- WordNet은 다양한 상황에서 같은 어휘라도 다르게 사용되는 어휘의 시멘틱 정보를 제공하며 이를 위해 가가각의 품사로 구성된 개별 단어를 synsetㅣ라는 개념을 이용하여 표현함
- NLTK의 감성 사저이 휼륭하긴 하지만 아쉽게도 예측 성능이 좋지는 못함. 그 때무에 실제 업무의 적용은 NLTK 패키지가 아닌 다른 감성 사전을 적용하는 것이 일반적임, NLTK를 포함한 대표적인 감성사전은 다음과 같음


### SentiWordNet:
wordnet의 synset 개념을 감성 분석에 적용하여 wordnet의 synset 별로 3가지 감성 점수 긍정, 부정, 객관성 지수를 할당함. 객관성은 긍정 부정과 완전히 반대되는 개념으로 단어가 감성과 관계없이 얼마나 객관적인지를 수치로 나타낸 것. 문장별로 단어들의 긍정 감성과 부정 감성 지수를 합산하여 최종 감성 지수를 계산하고 이에 기반해 감성이 긍정인지 부정인지를 결정함

## SentiWordNet을 이용한 감성 분석


In [11]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping grammars\basque_grammars.zip.
[nltk_data]   

[nltk_data]    | Downloading package nonbreaking_prefixes to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\nonbreaking_prefixes.zip.
[nltk_data]    | Downloading package nps_chat to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\nps_chat.zip.
[nltk_data]    | Downloading package omw to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package omw-1.4 to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    |   Package omw-1.4 is already up-to-date!
[nltk_data]    | Downloading package opinion_lexicon to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\opinion_lexicon.zip.
[nltk_data]    | Downloading package panlex_swadesh to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package paradigms to
[nltk_data]    | 

[nltk_data]    | Downloading package verbnet3 to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\verbnet3.zip.
[nltk_data]    | Downloading package webtext to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\webtext.zip.
[nltk_data]    | Downloading package wmt15_eval to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping models\wmt15_eval.zip.
[nltk_data]    | Downloading package word2vec_sample to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping models\word2vec_sample.zip.
[nltk_data]    | Downloading package wordnet to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    |   Package wordnet is already up-to-date!
[nltk_data]    | Downloading package wordnet2021 to
[nltk_data]    |     C:\Users\MYCOM\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package w

True

In [12]:
from nltk.corpus import wordnet as wn
term='present'
# present라는 단어로 wordnet의 synsets 생성
synsets=wn.synsets(term)
print(type(synsets))
print(len(synsets))
print(synsets)

<class 'list'>
18
[Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


In [13]:
# synset 객체가 가지는 여러가지 속성 살펴보기
for s in synsets:
    print('Synset name',s.name())
    print('POS',s.lexname()) # part of speech로 품사를 의미함
    print('Definition',s.definition())
    print('Lemmas',s.lemma_names())

Synset name present.n.01
POS noun.time
Definition the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas ['present', 'nowadays']
Synset name present.n.02
POS noun.possession
Definition something presented as a gift
Lemmas ['present']
Synset name present.n.03
POS noun.communication
Definition a verb tense that expresses actions or states at the time of speaking
Lemmas ['present', 'present_tense']
Synset name show.v.01
POS verb.perception
Definition give an exhibition of to an interested audience
Lemmas ['show', 'demo', 'exhibit', 'present', 'demonstrate']
Synset name present.v.02
POS verb.communication
Definition bring forward and present to the mind
Lemmas ['present', 'represent', 'lay_out']
Synset name stage.v.01
POS verb.creation
Definition perform (a play), especially on a stage
Lemmas ['stage', 'present', 'represent']
Synset name present.v.04
POS verb.possession
Definition hand over formally
Lemmas ['present', 'submit']
Synset

In [15]:
# WordNet은 어떤 어휘와 다른 어휘와의 관계를 유사도로 나타낼 수도 있음
# synset 객체를 단어별로 생성
tree=wn.synset('tree.n.01')
lion=wn.synset('lion.n.01')
tiger=wn.synset('tiger.n.02')
cat=wn.synset('cat.n.01')
dog=wn.synset('dog.n.01')
entities=[tree,lion,tiger,cat,dog]
similar=[]
entity_names=[entity.name().split('.')[0] for entity in entities]
# 단어별 synset 반복하면서 다른 단어와 유사도 측정
for entity in entities:
    sim=[round(entity.path_similarity(compared_entity),2)
        for compared_entity in entities]
    similar.append(sim)
# 개별 단어 synset과 다른 단어 와 유사도를 데이터 프레임 형태로 저장
sim_df=pd.DataFrame(similar,columns=entity_names,index=entity_names)
sim_df

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


In [16]:
# senti_synset 클래스를 리스트로 반환
from nltk.corpus import sentiwordnet as swn
senti_s=list(swn.senti_synsets('slow'))
print(type(senti_s))
print(len(senti_s))
print(senti_s)

<class 'list'>
11
[SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


In [17]:
# 두 단어의 감성 지수와 객관성 지수 나타내기
father=swn.senti_synset('father.n.01')
print(father.pos_score()) # 긍정감성 지수
print(father.neg_score()) # 부정감성지수
print(father.obj_score()) #객관성 지수

fab=swn.senti_synset('fabulous.a.01')
print(fab.pos_score()) # 긍정감성 지수
print(fab.neg_score()) # 부정감성 지수
print(fab.obj_score()) # 객관성 지수

0.0
0.0
1.0
0.875
0.125
0.0


## SentiWordNet을 이용한 영화 감상평 감성 분석
### Lexicon을 기반으로 수행하기
1. 문서를 문장 단위로 분해
2. 다시 문장을 단어 단위로 토큰화 하고 품사 태깅
3. 품사 태깅된 단어를 기반으로 synset 객체와 senti_synset 객체를 생성
4. senti_synset에서 긍정 부정 감성 지수를 구하고 이를 모두 합산해 특정 임계치 값 이상일 때는 긍정 감성으로, 그렇지 않을 때를 부정 감성으로 결정

In [18]:
from nltk.corpus import wordnet as wn
# 간단히 NLTK PennTreebank Tage를 기반으로 WordNet기반의 품사 tage로 변환
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB

In [19]:
def swn_polarity(text):
    
    from nltk.stem import WordNetLemmatizer
    from nltk.corpus import sentiwordnet as swn
    from nltk import sent_tokenize, word_tokenize, pos_tag

    # 감성 지수 초기화 
    sentiment = 0.0
    tokens_count = 0
    
    # 어근 추출 객체
    lemmatizer = WordNetLemmatizer()
    
    # 문장 토큰화
    raw_sentences = sent_tokenize(text) 
        
    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산 
    for raw_sentence in raw_sentences:
        
        # 각 문장별 단어 토큰화 후 품사 태깅 문장 추출 (단어와 품사 생성)
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        
        for word , tag in tagged_sentence:
            
            # WordNet 기반 품사 태깅
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN , wn.ADJ, wn.ADV):
                continue
                
            # 어근 추출
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            
            # 어근 추출한 단어, WordNet 기반 품사 태깅을 입력해 Synset 객체 생성 .. (1)
            synsets = wn.synsets(lemma , pos=wn_tag)
            if not synsets:
                continue
            
            # (1)에서 생성된 synset 객체의 첫 번째 의미 사용 (같은 품사여도 여러 의미 존재) .. (2)
            synset = synsets[0]
            
            # (2)를 이용해서 SentiSynset 객체 생성
            swn_synset = swn.senti_synset(synset.name())
            
            # 긍정 감성 지수 - 부정 감성 지수로 감성 지수 계산
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())           
            tokens_count += 1
    
    if not tokens_count:
        return 0
    
    # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
    if sentiment >= 0 :
        return 1
    
    return 0

In [20]:
# 각 문서별로 긍정/부정 예측
review_df['preds'] = review_df['review'].apply( lambda x : swn_polarity(x) )

y_target = review_df['sentiment'].values
preds = review_df['preds'].values

In [21]:
# 평가 지표 함수
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba_po=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    # auc = roc_auc_score(y_test, pred_proba_po)
   
    print("오차 행렬")
    print(confusion)
    print(f"정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}, F1: {f1:.4f}")

## VADER을 이용한 감성 분석

In [22]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(review_df['review'][0])

# dictionary로 반환
print(senti_scores)

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}


In [23]:
# 임계치별 긍부정 예측 함수
def vader_polarity(review, threshold = 0.1):
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    
    # VADER 객체로 감성 지수 산출
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    # 감성 지수가 threshold 보다 크거나 같으면 1, 그렇지 않으면 0
    agg_score = scores['compound']
    final_sentiment = 1 if agg_score >= threshold else 0
        
    return final_sentiment

In [24]:
# 각 문서별로 긍부정 예측
review_df['vader_preds'] = review_df['review'].apply( lambda x : vader_polarity(x, 0.1) )

y_target = review_df['sentiment'].values
vader_preds = review_df['vader_preds'].values

In [25]:
get_clf_eval(y_target, pred=vader_preds)

오차 행렬
[[ 6747  5753]
 [ 1858 10642]]
정확도: 0.6956, 정밀도: 0.6491, 재현율: 0.8514, F1: 0.7366
